In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [4]:
#Chargement de df_user
df_user=pd.read_csv('../../../../../dataset/data_regression/df_user.csv')

#Gestion des valeurs nulles
dico={}
for column in df_user.columns:
    if "nb_note_user" in str(column):
        dico[column]=0
    elif "avg_note_user" in str(column):
        dico[column]=df_user[column].mean()
    elif "avg_year_user" in str(column):
        dico[column]=df_user[column].mean()

df_user=df_user.fillna(value=dico)
del dico

In [5]:
#Chargement de df_note
df_note=pd.read_csv('../../../../../dataset/data_regression/df_note.csv', index_col=0)

In [6]:
#Join de df_user et df_note  -> df
df=df_note.merge(df_user, how='outer', on="userId")
del df_user
del df_note

In [8]:
#Join de df et df_movie 
df_movie=pd.read_csv('../../../../../dataset/data_regression/df_movie.csv')
df=df.merge(df_movie, how='outer', on="movieId_ref")
del df_movie
df=df.drop(["userId", "movieId_ref"], axis=1)

In [9]:
#preprocessing des données

for column in df.columns:
    scaler=MinMaxScaler()
    df[column]=scaler.fit_transform(pd.DataFrame(df[column]))
    
df.head()

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return sel

,rating,time_btw_vote,rating_user_avg,rating_user_nb,action_nb_note_user,action_avg_note_user,action_avg_year_user,adventure_nb_note_user,adventure_avg_note_user,adventure_avg_year_user,...,fantasy,history,horror,romance,sci-fi,thriller,war,western,averageRating,numVotes
0,0.666667,1.739614e-07,0.719228,0.239538,0.206349,0.700855,0.788462,0.317073,0.713675,0.803503,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.864198,0.377335
1,0.666667,2.193426e-07,0.719228,0.239538,0.206349,0.700855,0.788462,0.317073,0.713675,0.803503,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.691358,0.128565
2,0.666667,4.538122e-08,0.719228,0.239538,0.206349,0.700855,0.788462,0.317073,0.713675,0.803503,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.654321,0.010782
3,0.777778,6.807184e-08,0.719228,0.239538,0.206349,0.700855,0.788462,0.317073,0.713675,0.803503,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.555556,0.004174
4,0.555556,1.966520e-07,0.719228,0.239538,0.206349,0.700855,0.788462,0.317073,0.713675,0.803503,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.580247,0.015024


In [10]:
#Séparation des données train et test
data=df.drop(["rating"], axis=1)
target=df.rating
del df
X_train,X_test,y_train,y_test=train_test_split(data, target, test_size=0.05, train_size=0.2, random_state=222)
del data
del target

In [11]:
#Création du modèle de réseaux de neurones
model=Sequential()
model.add(Dense(76, input_dim=76, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
#Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

In [19]:
#Entrainement du modèle
model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

Epoch 1/100
 - 1s - loss: 0.0488
Epoch 2/100
 - 0s - loss: 0.0491
Epoch 3/100
 - 0s - loss: 0.0495
Epoch 4/100
 - 0s - loss: 0.0492
Epoch 5/100
 - 0s - loss: 0.0496
Epoch 6/100
 - 0s - loss: 0.0490
Epoch 7/100
 - 0s - loss: 0.0491
Epoch 8/100
 - 0s - loss: 0.0489
Epoch 9/100
 - 0s - loss: 0.0486
Epoch 10/100
 - 1s - loss: 0.0491
Epoch 11/100
 - 0s - loss: 0.0488
Epoch 12/100
 - 0s - loss: 0.0492
Epoch 13/100
 - 0s - loss: 0.0492
Epoch 14/100
 - 1s - loss: 0.0485
Epoch 15/100
 - 0s - loss: 0.0491
Epoch 16/100
 - 0s - loss: 0.0482
Epoch 17/100
 - 0s - loss: 0.0488
Epoch 18/100
 - 0s - loss: 0.0481
Epoch 19/100
 - 0s - loss: 0.0486
Epoch 20/100
 - 0s - loss: 0.0483
Epoch 21/100
 - 0s - loss: 0.0480
Epoch 22/100
 - 0s - loss: 0.0481
Epoch 23/100
 - 0s - loss: 0.0477
Epoch 24/100
 - 0s - loss: 0.0482
Epoch 25/100
 - 0s - loss: 0.0477
Epoch 26/100
 - 0s - loss: 0.0472
Epoch 27/100
 - 1s - loss: 0.0475
Epoch 28/100
 - 0s - loss: 0.0475
Epoch 29/100
 - 0s - loss: 0.0472
Epoch 30/100
 - 0s - lo

In [20]:
#Evaluation du modèle
y_pred=model.predict(X_test)
evaluate=model.evaluate(X_test, y_test)
print("perte : \t", evaluate[0])
print("erreur : \t", (1-evaluate[1])*100)

1181/1181 [==============================] - 0s 64us/step


IndexError: invalid index to scalar variable.

In [21]:
evaluate

0.0535096824295333